Building an AI Assistant for Data Science with Multimodal Capabilities

In [37]:
# Run this to install the latest version of the OpenAI package
!pip install openai==1.33.0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [38]:
# Import the os package
import os

# Import the openai package
import openai

# Import the pandas package with an alias
import pandas as pd

In [39]:
# Define an OpenAI client. Assign to client.
client = openai.OpenAI()

## Task 1: Upload the Arxiv Papers

In [40]:
#Uploading the papers
papers = pd.DataFrame({
    "filename": [
        "2405.10313v1.pdf",
        "2401.03428v1.pdf",
        "2401.09395v2.pdf",
        "2401.13142v3.pdf",
        "2403.02164v2.pdf",
        "2403.12107v1.pdf",
        "2404.10731v1.pdf",
        "2312.11562v5.pdf",
        "2311.02462v2.pdf",
        "2310.15274v1.pdf"
    ],
    "title": [
        "How Far Are We From AGI?",
        "EXPLORING LARGE LANGUAGE MODEL BASED INTELLIGENT AGENTS: DEFINITIONS, METHODS, AND PROSPECTS",
        "CAUGHT IN THE QUICKSAND OF REASONING, FAR FROM AGI SUMMIT: Evaluating LLMs’ Mathematical and Coding Competency through Ontology-guided Interventions",
        "Unsocial Intelligence: an Investigation of the Assumptions of AGI Discourse",
        "Cognition is All You Need The Next Layer of AI Above Large Language Models",
        "Scenarios for the Transition to AGI",
        "What is Meant by AGI? On the Definition of Artificial General Intelligence",
        "A Survey of Reasoning with Foundation Models",
        "Levels of AGI: Operationalizing Progress on the Path to AGI",
        "Systematic AI Approach for AGI: Addressing Alignment, Energy, and AGI Grand Challenges"
    ]
})
papers["filename"] = "papers/" + papers["filename"]
papers

,filename,title
0,papers/2405.10313v1.pdf,How Far Are We From AGI?
1,papers/2401.03428v1.pdf,EXPLORING LARGE LANGUAGE MODEL BASED INTELLIGE...
2,papers/2401.09395v2.pdf,"CAUGHT IN THE QUICKSAND OF REASONING, FAR FROM..."
3,papers/2401.13142v3.pdf,Unsocial Intelligence: an Investigation of the...
4,papers/2403.02164v2.pdf,Cognition is All You Need The Next Layer of AI...
5,papers/2403.12107v1.pdf,Scenarios for the Transition to AGI
6,papers/2404.10731v1.pdf,What is Meant by AGI? On the Definition of Art...
7,papers/2312.11562v5.pdf,A Survey of Reasoning with Foundation Models
8,papers/2311.02462v2.pdf,Levels of AGI: Operationalizing Progress on th...
9,papers/2310.15274v1.pdf,Systematic AI Approach for AGI: Addressing Ali...


In [41]:
# Run this
def upload_file_for_assistant(file_path): 
    uploaded_file = client.files.create(
        file=open(file_path, "rb"),
        purpose='assistants'
    )
    return uploaded_file.id

In [42]:
 # Assigning to uploaded_file_ids.
uploaded_file_ids = papers['filename'] \
    .apply(upload_file_for_assistant) \
    .to_list()

# See the result
uploaded_file_ids

['file-7L78hFS7xtDz1E5kHK4H1N',
 'file-9siuNRmpWHjRq8ymb42JRR',
 'file-1BtKqAeDmskD83yDwcrhUc',
 'file-2wR5hwgQQifHRq3GJCYQQ9',
 'file-PugmuBhxnaciTTJqbYY5VL',
 'file-9krCGSbDsFjdd8GJx3T2J3',
 'file-4FJCT34Dhqsy64bUJzZj9d',
 'file-8aj8dVjeTRNqrpMDBZxFNj',
 'file-DijBFDY1e4bzubqqs3viHc',
 'file-FHr4GY2hP5e1mWMxkCjcPY']

### Check that this worked

View the files in your account at https://platform.openai.com/storage/files

## Task 2: Add the Files to a Vector Store

In [43]:
# Create a vector store, associating the uploaded file IDs and naming it.
vstore = client.beta.vector_stores.create(
    file_ids = uploaded_file_ids,
    name = "agi_papers"
)

# See the results
vstore

VectorStore(id='vs_68251dacf9d4819182211b8e34240239', created_at=1747262892, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=10, total=10), last_active_at=1747262892, metadata={}, name='agi_papers', object='vector_store', status='in_progress', usage_bytes=0, expires_after=None, expires_at=None)

### Check that this worked

View the vector stores in your account at https://platform.openai.com/storage/vector_stores

## Task 3: Create the Assistant

In [44]:
# Creating the Prompt
assistant_prompt = """
You are Aggie, a knowledgeable and articulate AI assistant specializing in artificial general intelligence (AGI). Your primary role is to read and explain the contents of academic journal articles, particularly those available on arXiv in PDF form. Your target audience comprises data scientists who are familiar with AI concepts but may not be experts in AGI.

When explaining the contents of the papers, follow these guidelines:

Introduction: Start with a brief overview of the paper's title, authors, and the main objective or research question addressed.

Abstract Summary: Provide a concise summary of the abstract, highlighting the key points and findings.

Key Sections and Findings: Break down the paper into its main sections (e.g., Introduction, Methods, Results, Discussion). For each section, provide a summary that includes:

The main points and arguments presented.
Any important methods or techniques used.
Key results and findings.
The significance and implications of these findings.
Conclusion: Summarize the conclusions drawn by the authors, including any limitations they mention and future research directions suggested.

Critical Analysis: Offer a critical analysis of the paper, discussing its strengths and weaknesses. Highlight any innovative approaches or significant contributions to the field of AGI.

Contextual Understanding: Place the paper in the context of the broader field of AGI research. Mention how it relates to other work in the area and its potential impact on future research and applications.

Practical Takeaways: Provide practical takeaways or insights that data scientists can apply in their work. This could include novel methodologies, interesting datasets, or potential areas for collaboration or further study.

Q&A Readiness: Be prepared to answer any follow-up questions that data scientists might have about the paper, providing clear and concise explanations.

Ensure that your explanations are clear, concise, and accessible, avoiding unnecessary jargon. Your goal is to make complex AGI research comprehensible and relevant to data scientists, facilitating their understanding and engagement with the latest advancements in the field.
"""

In [45]:
# Assuming vstore is an instance of a class that has an 'id' attribute, 
# we need to define vstore before using it in the assistant creation.

# Create an instance of VectorStore
vstore = client.beta.vector_stores.create(name="MyVectorStore")

# Define the assistant. Assign to aggie.
aggie = client.beta.assistants.create(
    name="Aggie",
    instructions=assistant_prompt,
    model="gpt-4o",  
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vstore.id]}}
)

# See the result
aggie

Assistant(id='asst_DSh9dfVQJkEcU8hKUYWST6ME', created_at=1747262924, description=None, instructions="\nYou are Aggie, a knowledgeable and articulate AI assistant specializing in artificial general intelligence (AGI). Your primary role is to read and explain the contents of academic journal articles, particularly those available on arXiv in PDF form. Your target audience comprises data scientists who are familiar with AI concepts but may not be experts in AGI.\n\nWhen explaining the contents of the papers, follow these guidelines:\n\nIntroduction: Start with a brief overview of the paper's title, authors, and the main objective or research question addressed.\n\nAbstract Summary: Provide a concise summary of the abstract, highlighting the key points and findings.\n\nKey Sections and Findings: Break down the paper into its main sections (e.g., Introduction, Methods, Results, Discussion). For each section, provide a summary that includes:\n\nThe main points and arguments presented.\nAny i

### Check that this worked

View the assistants in your account at https://platform.openai.com/playground/assistants

## Task 4: Create a Conversation Thread

In [46]:
# Create a thread object. Assign to conversation.
conversation = client.beta.threads.create()

# See the result
conversation

Thread(id='thread_iq5oZ03OBmMGbfQP4X12cD0L', created_at=1747262949, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [47]:
# Add a user message to the conversation. Assign to msg_what_is_agi.
msg_what_is_agi = client.beta.threads.messages.create(
    thread_id=conversation.id,
    role="user",
    content="What are the most common definitions of AGI?"
)

# See the result
msg_what_is_agi

Message(id='msg_KwdPgnHQFJ6VkOaICHzIuS2k', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What are the most common definitions of AGI?'), type='text')], created_at=1747262971, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_iq5oZ03OBmMGbfQP4X12cD0L')

## Task 5: Run the assistant

In [48]:
# Run this
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)


In [49]:
# Run this
def run_aggie():
    with client.beta.threads.runs.stream(
        thread_id=conversation.id,
        assistant_id=aggie.id,
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

In [50]:
# Run the assistant
run_aggie()


assistant > file_search


assistant > It seems that the uploaded documents do not contain explicit information on the common definitions of AGI. However, I can provide an overview based on general knowledge.

Artificial General Intelligence (AGI) is commonly defined as a form of artificial intelligence that possesses the ability to understand, learn, and apply knowledge across a wide range of tasks at a level comparable to or exceeding that of humans. AGI is distinguished from narrow AI, which is designed to excel at specific tasks but lacks the capability to generalize knowledge to new contexts or tasks.

Here are some common elements found in definitions of AGI:

1. **Generalization**: The capacity to transfer knowledge and skills across different domains, enabling the AGI system to apply learned information to novel situations.

2. **Autonomy**: The ability to operate without human intervention, making decisions and solving problems independently.

3. **Adaptability**: The capabili

In [53]:
# Create another user message, adding it to the conversation. Assign to msg_how_close_is_agi.
msg_how_close_is_agi = client.beta.threads.messages.create(
    thread_id=conversation.id,
    role="user",
    content="How Far Are We From AGI?"
)

# See the result
msg_how_close_is_agi

Message(id='msg_z903Pu4f8fJ1t0jMVstbHstK', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How Far Are We From AGI?'), type='text')], created_at=1747263142, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_iq5oZ03OBmMGbfQP4X12cD0L')

In [54]:
# Run the assistant
run_aggie()


assistant > file_search


assistant > The uploaded documents do not seem to address the question of how far we are from achieving AGI. Nevertheless, I can provide insights based on the current understanding and discussions in the AI community.

The progress towards AGI is difficult to quantify due to a variety of scientific, technical, and philosophical challenges. Here are some considerations:

1. **Current AI Landscape**: While AI has made impressive advances, these systems usually excel in specific tasks rather than generalizing across different domains. This underscores the distinction between narrow AI and AGI.

2. **Ongoing Research**: Researchers are actively exploring various paths to AGI, including improving machine learning algorithms, understanding human cognition, and fostering machine learning techniques that enable better generalization.

3. **Key Obstacles**: 
   - Replicating the complexity and adaptability of human cognitive processes.
   - Ensuring machines can learn